# [DBA Mastery](http://dbamastery.com)
> **Just another witchcraft and wizardry site and DBA tips …**

# Upgrading a SQL Server docker container using Python in ADS Notebooks

This Azure Data Studio Notebook uses the Docker Python API to manage SQL Server container created in Docker. In this example I upgrade a SQL Server instance (container) from SQL Server 2017 CU16 to CU17.

For more information about the Docker SDK for Python, take a look at my SQL Server Central articles:
* [Managing SQL Server containers using Docker SDK for Python - Part 1](https://www.sqlservercentral.com/articles/managing-sql-server-containers-using-docker-sdk-for-python-part-1)
* [Managing SQL Server containers using Docker SDK for Python - Part 2](https://www.sqlservercentral.com/articles/managing-sql-server-containers-using-docker-sdk-for-python-part-2)

These are the steps to be perform on this ADS Notebook:

1. Checking SQL Server images in MCR **(optional)**  
    * **Ubuntu based images**  
    * **RHEL based images** 
2. Pull images **(optional)**
3. Check images **(optional)**
4. Create CU16 container (with volume)
5. Check container status
6. Connect to CU16 and get version
7. Restore WWI database from backup
8. Stop and upgrade CU16 to CU17
9. Check SQL Server errorlog
10. Connect to CU17 and get version

# 1. Checking SQL Server images in MCR - Ubuntu based (Python)



In [ ]:

# Importing HTTP requests library
import requests

# Listing available images at MCR for Ubuntu
ubuntu_mcr_images = requests.get('https://mcr.microsoft.com/v2/mssql/server/tags/list/')

# Displaying HTTP GET results
print ('Ubuntu based images:')
ubuntu_mcr_images.json()

# 1. Checking SQL Server images in MCR - RHEL based (Python)

In [ ]:

# Importing HTTP requests library
import requests

# Listing available images at MCR for RHEL
rhel_mcr_images = requests.get('https://mcr.microsoft.com/v2/mssql/rhel/server/tags/list/')

# Displaying HTTP GET results
print ('RHEL based images:')
rhel_mcr_images.json()

# 2. Pull images (Python)

In [ ]:
# Importing docker library
import docker
client = docker.from_env()

client.images.pull('mcr.microsoft.com/mssql/server:2017-CU16-ubuntu')
print ('The SQL Server 2017 - Ubuntu CU16 image has been pulled')
client.images.pull('mcr.microsoft.com/mssql/server:2017-CU17-ubuntu')
print ('The SQL Server 2017 - Ubuntu CU17 image has been pulled')

# 3. Check images (Python)

In [ ]:

# Listing local images (Docker cache)
client.images.list()


# 4. Create CU16 container (Python)

In [ ]:
# Checking if existing \ Cleaning up
try:
        container = client.containers.get('PythonSQL_CU16')
        container.stop()
        container.remove()
        print ('The existing PythonSQL_CU16 container was removed ...')
except:
        print ("The PythonSQL_CU16 container does not exists ...")

# Creating Python_CU16 container
client.containers.run(
        'mcr.microsoft.com/mssql/server:2017-CU16-ubuntu',
        detach=True,
        name="PythonSQL_CU16",
        hostname="PythonSQL_CU16",
        environment={"ACCEPT_EULA":"Y","MSSQL_SA_PASSWORD":"P1th0nR0ck5"},
        volumes= 
        {'vlm_VersionUpgrade': {'bind': '/var/opt/mssql', 'mode': 'rw'},
        '/Users/carlos/Documents/DBA Mastery/Shared': {'bind': '/Shared', 'mode': 'rw'}},
        ports={'1433/tcp': 1401}
)
print ("The PythonSQL_CU16 container was succesfully created")

# 5. Check container status (Python)

In [ ]:
# Importing docker library
import docker

# Setting context to my container
container = client.containers.get('PythonSQL_CU16')

# Checking status of PythonSQL container
print ('My container: ' + container.name + ' is in ' + container.status + ' state.')

# Checking status of all containers
client.containers.list(all=True, filters={"status":"running"})

# 6. Connect to CU16 and get version (SQL)

In [ ]:
-- Checking SQL Server instance information
-- 14.0.3223.3 = CU16
-- 14.0.3238.1 = CU17
SELECT
    SERVERPROPERTY('MachineName') AS [Computer Name],
    SERVERPROPERTY('ServerName') AS [Instance Name],
    SERVERPROPERTY('ProductVersion') AS [Product Version],
    SERVERPROPERTY('ProductUpdateLevel') AS [CU],
    RIGHT(@@version, LEN(@@version)- 3 -charindex (' ON ', @@VERSION)) [OS Version],
    SERVERPROPERTY ('Edition') AS [Edition];

# 7. Restore WWI database from backup (SQL)

In [ ]:
-- Checking existing databases
SELECT name FROM sys.databases;

-- Restore WWI backup
RESTORE DATABASE WWI FROM DISK = '/Shared/wwi.bak' WITH
MOVE 'WWI_Primary' TO '/var/opt/mssql/data/WideWorldImporters.mdf',
MOVE 'WWI_UserData' TO '/var/opt/mssql/data/WideWorldImporters_userdata.ndf',
MOVE 'WWI_Log' TO '/var/opt/mssql/data/WideWorldImporters.ldf', 
MOVE 'WWI_InMemory_Data_1' TO '/var/opt/mssql/data/WideWorldImporters_InMemory_Data_1';

-- Looking for WWI database
SELECT name FROM sys.databases;

# 8. Stop and upgrade CU16 to CU17 (Python)

In [ ]:
# Importing docker library
import docker
client = docker.from_env()

# Setting context to my container
container = client.containers.get('PythonSQL_CU16')

# Stopping PythonSQL
container.stop()

# Creating Python_CU17 container (upgrade)
client.containers.run(
        'mcr.microsoft.com/mssql/server:2017-CU17-ubuntu',
        detach=True,
        name="PythonSQL_CU17",
        hostname="PythonSQL_CU17",
        environment={"ACCEPT_EULA":"Y","MSSQL_SA_PASSWORD":"P1th0nR0ck$!"},
        volumes= 
        {'vlm_VersionUpgrade': {'bind': '/var/opt/mssql', 'mode': 'rw'},
        '/Users/carlos/Documents/DBA Mastery/Shared': {'bind': '/Shared', 'mode': 'rw'}},
        ports={'1433/tcp': 1401}
)
print ("The PythonSQL_CU17 container was succesfully created")

# 9. Check SQL Server errorlog (Python)

In [ ]:
# Setting context to my container
container = client.containers.get('PythonSQL_CU17')

# Checking the log
for line in container.logs(stream=True):
    print (line.strip())


# 10. Connect to CU17 and get version (SQL)

In [ ]:
-- Checking SQL Server instance information
-- 14.0.3048.4 = CU16
-- 14.0.3076.1 = CU17
SELECT
    SERVERPROPERTY('MachineName') AS [Computer Name],
    SERVERPROPERTY('ServerNameA') AS [Instance Name],
    SERVERPROPERTY('ProductVersion') AS [Product Version],
    SERVERPROPERTY('ProductUpdateLevel') AS [CU],
    RIGHT(@@version, LEN(@@version)- 3 -charindex (' ON ', @@VERSION)) [OS Version],
    SERVERPROPERTY ('Edition') AS [Edition];

-- Looking for WWI database
SELECT name FROM sys.databases;

# Questions or comments?

>If you want to know more details about this ADS Notebook, feel free to reach me out at:

- Twitter:&nbsp;&nbsp;&nbsp;&nbsp;[@dbamastery](https://twitter.com/dbamastery)  
- GitHub:&nbsp;&nbsp;&nbsp;&nbsp;[dbamaster](https://github.com/dbamaster)  
- Email:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<crobles@dbamastery.com> 
- Blog:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;http://dbamastery.com
